### Test case

3<br>
0 0<br>
0 3<br>
3 3<br>
// 10.24
<br>
<br>
4<br>
0 0<br>
4 3<br>
4 0<br>
0 3<br>
// 14.00
<br>
<br>
8<br>
0 2.05<br>
3.414213562373095 3.4642135623730947<br>
0.5857864376269049 0.6357864376269047<br>
0.5857864376269049 3.4642135623730947<br>
2 0<br>
4.05 2.05<br>
2 4.10<br>
3.414213562373095 0.6357864376269047<br>
// 12.36

In [2]:
import re
import math
import numpy as np
import pandas as pd

def convert_to_dist(filename):
    with open(filename) as f:
        content = f.readlines()
    pattern = re.compile('\d+\.?\d*')
    matches = [re.findall(pattern, x) for x in content]  # [['3'], ['0', '0'], ['0', '3'], ['3', '3']]
    idx_len = int(matches[0][0])  
    idx_lst = [i+1 for i in range(idx_len)]
    df = pd.DataFrame(np.zeros((idx_len, idx_len)), index=idx_lst, columns=idx_lst)
    
    nodes = [[float(x[0]), float(x[1])] for x in matches[1:]]
    for i, x in enumerate(nodes):
        for j, y in enumerate(nodes):
            df[j+1][i+1] = math.sqrt(math.pow((x[0]-y[0]), 2) + math.pow((x[1]-y[1]), 2))
            
    return df, idx_lst

In [195]:
df, idx_lst = convert_to_dist('datasets/tsp_test2.txt')
df

,1,2,3,4
1,0.0,5.0,4.0,3.0
2,5.0,0.0,3.0,4.0
3,4.0,3.0,0.0,5.0
4,3.0,4.0,5.0,0.0


In [243]:
# naive
import itertools

df, idx_lst = convert_to_dist('datasets/tsp_test2.txt')
perm = itertools.permutations(idx_lst)
dist_lst = []
for n in perm:
    dist = 0
    for idx in range(len(n)-1):
        dist += df[n[idx+1]][n[idx]]
    dist += df[n[-1]][n[0]]
    dist_lst.append(dist)

min(dist_lst)

14.0

In [263]:
df, idx_lst = convert_to_dist('datasets/tsp_test2.txt')
df

,1,2,3,4
1,0.0,5.0,4.0,3.0
2,5.0,0.0,3.0,4.0
3,4.0,3.0,0.0,5.0
4,3.0,4.0,5.0,0.0


In [3]:
# dynamic programming O(n^2*2^n) https://www.geeksforgeeks.org/travelling-salesman-problem-set-1/

import re
import math
import numpy as np
import pandas as pd

def convert_to_dist(filename):
    with open(filename) as f:
        content = f.readlines()
    pattern = re.compile('\d+\.?\d*')
    matches = [re.findall(pattern, x) for x in content]  # [['3'], ['0', '0'], ['0', '3'], ['3', '3']]
    idx_len = int(matches[0][0])  
    idx_lst = [i+1 for i in range(idx_len)]
    df = pd.DataFrame(np.zeros((idx_len, idx_len)), index=idx_lst, columns=idx_lst)
    
    nodes = [[float(x[0]), float(x[1])] for x in matches[1:]]
    for i, x in enumerate(nodes):
        for j, y in enumerate(nodes):
            df[j+1][i+1] = math.sqrt(math.pow((x[0]-y[0]), 2) + math.pow((x[1]-y[1]), 2))
            
    return df, idx_lst

df, idx_lst = convert_to_dist('datasets/tsp_test3.txt')

df

,1,2,3,4,5,6,7,8
1,0.000000,3.695518,1.530734,1.530734,2.864001,4.050000,2.864001,3.695518
2,3.695518,0.000000,4.000000,2.828427,3.741761,1.550556,1.550556,2.828427
3,1.530734,4.000000,0.000000,2.828427,1.550556,3.741761,3.741761,2.828427
4,1.530734,2.828427,2.828427,0.000000,3.741761,3.741761,1.550556,4.000000
5,2.864001,3.741761,1.550556,3.741761,0.000000,2.899138,4.100000,1.550556
6,4.050000,1.550556,3.741761,3.741761,2.899138,0.000000,2.899138,1.550556
7,2.864001,1.550556,3.741761,1.550556,4.100000,2.899138,0.000000,3.741761
8,3.695518,2.828427,2.828427,4.000000,1.550556,1.550556,3.741761,0.000000


In [4]:
def calc_cost(start, set_, end):
    if len(set_) == 1:
        temp = set_.pop()
        return df[start][temp] + df[temp][end]
    else:
        all_ = []
        for idx in (set_):
            new_set = set_.copy()
            new_set.remove(idx)
            cost = calc_cost(start, new_set, idx) + df[idx][end]
            all_.append(cost)
        return min(all_)


def tsp(start, end):
    lst = idx_lst.copy()
    lst.remove(start)
    lst.remove(end)
    cost = calc_cost(start, set(lst), end)
    total = cost + df[end][start]
    return total
            

def main():
    min_dist = 9999999999
    for i in range(1, len(idx_lst)+1):
        for j in [x for x in idx_lst if x > i]:
            cost = tsp(i, j)
            if cost < min_dist:
                min_dist = cost
    print("min_dist: ")
    print(min_dist)


main()

min_dist: 
12.364804397553604


In [ ]:
# command line example
# python tsp.py '/Users/yuqinglu/E_Learning/Coursera_Algorithms/datasets/tsp_test3.txt'
'''
    Let A = 2D array, indexed by subsets S in {1,2,...,n} that contain 1 and destinations j in {1,2,...n}
	Base case: A[S,1] = 0 if S={1}  or infinity otherwise
	for m=2,3,4,...,n:
    for each set S in {1,2,...,n} of size m that contains 1:
    for each j in S, j!=1:
    A[S,j]=min_{k in S and k!=j}  { A[S-{j},k]+Ckj }
	return min_{j=2,...,m} {A[{1,2,3,...,n},j]+Cj1}
'''

import numpy as np
import sys

LARGENUMBER =  1e20

def builddist(ncity,x,y):
    distance = np.zeros((ncity,ncity))
    x = x-min(x)
    y = y-min(y)
    for i in range(ncity):
        for j in range(i,ncity):
            distance[i,j]=distance[j,i]=np.sqrt((x[i]-x[j])**2+(y[i]-y[j])**2)
    return distance


# build set ordered as increasing number of elements
# index+1 ->  binary representation of the set:
#             1 - in set, 0 - not in set
def buildset(nelement):
    nset = 2**(nelement-1)
    orderset = [bin(i).count("1") for i in range(nset)]
    orderset = np.argsort(orderset)
    return orderset

# return combination number
def comb(n,k):
    up = 1
    for i in range(k):
        up *= (n-i)
    for i in range(k):
        up /= i+1
    return up

# check the order of the set is correct
def check(n):
    od=buildset(n)
    index=0
    for i in range(n-1):
        nelement=comb(n-1,i)
        print '--',i,nelement
        for j in range(nelement):
            print j+index,od[j+index],bin(od[j+index]).count("1")
        index+=nelement

# solve the TSP.
def search(dist,od,ncity):
    nset = 2**(ncity-1)
    A = np.zeros((nset,ncity))+LARGENUMBER
    A[0,0] = 0
    index = 1
    firstpart = 1
    oldelement = 1
    for m in range(1,ncity): # loop all subproblem size
        nelement=np.int(comb(ncity-1,m))
        print "processing ..",m,'/',ncity," for ",nelement,"cases"
        process = 10
        for i in range(nelement): # loop all subset S with size m
            if (i*100)/nelement > process:
                print "      finish ",process,'%'
                process+=10
            for j in range(1,ncity):
                if bin(od[index]).zfill(ncity+1)[-j] == '1': # check j in S
                    temp = np.zeros(ncity)+LARGENUMBER
                    #print nelement,od[0:oldelement]
                    target = np.where(od[0:oldelement] == od[index]-2**(j-1)) # S-{j} case
                    #print target,od[0:oldelement],od,od[index]-2**(j-1)
                    temp[0] = A[target,0]+dist[0,j]
                    for k in range(1,ncity):
                        if bin(od[index]).zfill(ncity+1)[-k] == '1': # check k in S
                            temp[k]=A[target,k]+dist[k,j]
                    A[index,j]=min(temp)
            index+=1
        # remove excess part
        index -= firstpart
        A=np.delete(A,np.s_[0:firstpart],0)
        od=np.delete(od,np.s_[0:firstpart],0)
        firstpart = nelement
        oldelement = nelement
    temp = np.zeros(ncity)+LARGENUMBER
    for i in range(1,ncity):
        temp[i]=A[-1,i]+dist[0,i]
    return min(temp)


if __name__ == "__main__":
    ncity = np.fromfile(sys.argv[1],count=1,sep=' ')
    x,y = np.loadtxt(sys.argv[1],skiprows=1,unpack=True)
    dist=builddist(ncity,x,y)
    od=buildset(ncity)
    print search(dist,od,ncity)

In [ ]:
import math
class TSP:
    def __init__(self,filename):
        self.G={}
        self.dist={}
        self.prev_setS=set()
        self.curr_setS=set()
        self.prev_A={}
        self.curr_A={}
        self.inputfile=filename
        self.min_dist=float("inf")
    def readinput(self):
        with open(self.inputfile) as f:
            c=0
            for line in f:
                v=line.split()
                v=[float(i) for i in v]
                if len(v)>1:
                    self.G[c]={}
                    self.G[c]['x']=v[0]
                    self.G[c]['y']=v[1]
                    c+=1
    def evalaute_distace(self):
        num_of_cities=len(self.G.keys())
        for city1 in range(0,num_of_cities-1):
            for city2 in range(city1+1,num_of_cities):
                dist=math.sqrt((self.G[city1]['x']-self.G[city2]['x'])**2 +\
                        (self.G[city1]['y']-self.G[city2]['y'])**2)
                self.dist[(city1,city2)]=self.dist[(city2,city1)]=dist

    def setS(self,cardinatlity):
        if self.prev_setS==set():
            self.prev_setS={1<<x for x in self.G.keys()}
        else:
            self.prev_setS=self.curr_setS
        self.curr_setS=set()
        for seta in self.prev_setS:
            for i in range(0,len(self.G.keys())):
                    setb=seta|(1<<i)
                    if bin(setb).count('1') == cardinatlity:
                        self.curr_setS.add(setb)
        return self.curr_setS

    def tsp_dist(self):
        #base case
        for m in range(2,len(self.G.keys())+1):
            self.prev_A=self.curr_A
            self.prev_A[(1,0)]=0
            self.curr_A={}
            print('current cardinality is ',m)
            for S in self.setS(m):
                for j in self.G.keys():
                    temp=float("inf")
                    if 1<<j & S:
                        #A[s,j]=min(A[s-j,k]+dist_kj
                        for k in self.G.keys():
                            if (1<<k & S) and (k!=j):
                                S_j=S^(1<<j)
                                temp=min(self.prev_A.get((S_j,k),float("inf")) + self.dist[(k,j)],temp)
                        self.curr_A[S,j]=temp
        #last hop

        S=2**(len(self.G.keys()))-1
        for j in range(1,len(self.G.keys())):
            self.min_dist=min(self.curr_A[S,j]+self.dist[(j,0)],self.min_dist)
        print(self.min_dist)

#s=TSP('city.txt')
#s.readinput()
#s.evalaute_distace()
#s.tsp_dist()